In [87]:
'''
YOUR TASKS:

1. SPECIFY THE URL OR PATH TO YOUR DATA, REMEMBER - YOU FIRST HAVE TO DOWNLOAD IT.
2. CLICK RUN ALL OR RESTART AND RUN ALL BUTTON TO RUN THE CODE. THIS CODE RUNS IN A CHRONOLOGICAL MANNER - THE TOP BLOCKS MUST PRECEED THE BOTTOM ONES,
                                                                                                            i.e. don't run stuff from the bottom first, otherwise the code won't work properly.
3. NEAR THE END OF THE CODE, YOU WILL SEE YOUR EPOCH METRICS, AS I STATED IN CLASS, YOU WANT THE ACCURACY TO APPROACH 100% =1.0 AND THE LOSS TO APPROACH 0
4. IN CASE IF YOUR MODEL IS STRUGGLING TO REACH THESE RESULTS AND IS INSTEAD STUCK AT A LOW ACCURACY, NAVIGATE TO THE BLOCK OF CODE WITH HYPERPARAMETERS AND CHANGE THESE NUMBERS,
THEN, TRY RUNNING THE CODE AGAIN AND SEE HOW YOUR MODEL IS PERFORMING; REPEAT UNTIL YOU REACH A PLAUSIBLE RESULT (ONE THAT IS BETTER THAN A COIN TOSS IN TERMS OF PREDICTION ACCURACY IN THE WORST CASE)
5. SAVE YOUR RESULTS => THE CODE INCLUDES A BLOCK WHICH WILL SAVE YOUR MODEL TO A .h5 FILE,
 - KEEP THE OUTPUTS OF THIS NOTEBOOK AS WELL, YOU WILL WANT TO DEMONSTRATE HOW THE TRAINING PROCESS WENT.
 - BE MINDFUL OF THE TIME YOU HAVE IF YOU CHOOSE TO USE GOOGLE COLLABORATORY INSTEAD OF YOUR OWN MACHINE AS THEIR VMs ONLY OFFER YOU A COUPLE OF HOURS OF USAGE.
'''

"\nYOUR TASKS:\n\n1. SPECIFY THE URL OR PATH TO YOUR DATA, REMEMBER - YOU FIRST HAVE TO DOWNLOAD IT.\n2. CLICK RUN ALL OR RESTART AND RUN ALL BUTTON TO RUN THE CODE. THIS CODE RUNS IN A CHRONOLOGICAL MANNER - THE TOP BLOCKS MUST PRECEED THE BOTTOM ONES, \n                                                                                                            i.e. don't run stuff from the bottom first, otherwise the code won't work properly.\n3. NEAR THE END OF THE CODE, YOU WILL SEE YOUR EPOCH METRICS, AS I STATED IN CLASS, YOU WANT THE ACCURACY TO APPROACH 100% =1.0 AND THE LOSS TO APPROACH 0\n4. IN CASE IF YOUR MODEL IS STRUGGLING TO REACH THESE RESULTS AND IS INSTEAD STUCK AT A LOW ACCURACY, NAVIGATE TO THE BLOCK OF CODE WITH HYPERPARAMETERS AND CHANGE THESE NUMBERS,\nTHEN, TRY RUNNING THE CODE AGAIN AND SEE HOW YOUR MODEL IS PERFORMING; REPEAT UNTIL YOU REACH A PLAUSIBLE RESULT (ONE THAT IS BETTER THAN A COIN TOSS IN TERMS OF PREDICTION ACCURACY IN THE WORST CASE)\n5. SAVE YOUR 

In [88]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import time

In [89]:
# Function to read the data
def read_data(filename):
    data = pd.read_csv(filename)

    # Extract features
    features = data.iloc[:,1 :-1].values

    # Extract and encode labels
    label_encoder = LabelEncoder()
    data.iloc[:, -1] = label_encoder.fit_transform(data.iloc[:, -1])
    labels = data.iloc[:, -1].values

    return features, labels
# Read the data
data_url = 'https://raw.githubusercontent.com/camilleb2700/ENGIN-442/main/user_12.csv'# enter your data URL or PATH here in enclosed in " " IF YOUR NOTEBOOK IS IN THE SAME FOLDER AS data.csv, which is your data you downloaded, then this will run as it is.
                    # otherwise, you have to specify a different path or url
x, y = read_data(data_url)

<ipython-input-89-f71415f27b7e>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, -1] = label_encoder.fit_transform(data.iloc[:, -1])


In [90]:
df = pd.read_csv(data_url) ## THIS AND THE BLOCK OF CODE IMMEDIATELY AFTER THIS ARE OPTIONAL, THIS IS INTENDED TO HELP YOU VISUALIZE YOUR DATASET

In [91]:
df # AS STATED ABOVE, YOU MAY REMOVE THIS AND THE ABOVE BLOCKS OF CODE IF YOU LIKE, THIS WILL HAVE NO

,Time (Seconds),x-rotation,y-rotation,z-rotation,x-position,y-position,z-position,label
0,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
1,0.008333,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
2,0.016667,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
3,0.025000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
4,0.033333,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,Sit_VidCall_Phone
...,...,...,...,...,...,...,...,...
81195,30.350000,145.497528,1.136816,-100.203384,0.476438,1.215163,0.705384,Walk_WebBrowse_Tablet
81196,30.358333,145.435791,0.323997,-100.612747,0.477562,1.214183,0.704687,Walk_WebBrowse_Tablet
81197,30.366667,145.658401,-0.436725,-101.451706,0.478550,1.213255,0.704206,Walk_WebBrowse_Tablet
81198,30.375000,146.021698,-1.215853,-102.191917,0.479562,1.212412,0.703331,Walk_WebBrowse_Tablet


In [92]:
x.shape ### IN THE FOLLOWING PARTS OF THE CODE, MAKE SURE THAT EVERYTING IS LOADED CORRECTLY, YOU SHOULD HAVE A DATASET WITH 22 CLASSES, (n-rows, 6 columns, later another dimension will be added here)

(81200, 6)

In [93]:
y.shape

(81200,)

In [94]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21])

In [95]:
x[0]

array([-0., -0., -0., -0.,  0.,  0.])

In [96]:
y[0] # these were sample rows from the data for your to inspect, you should see a 6 by 1 vector above and a single value here

0

In [97]:
# Split the data into training, validation, and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [98]:
# Reshape the data
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [99]:
# Determine the input shape and number of classes
input_shape = (x_train.shape[1], 1)
num_classes = len(np.unique(y))

In [100]:
num_classes # must have 22

22

In [101]:
input_shape

(6, 1)

In [102]:
x_train.shape

(58464, 6, 1)

In [103]:
y_train.shape

(58464,)

In [104]:
x_train[0]

array([[  5.603017],
       [ -3.832383],
       [144.152039],
       [  1.588381],
       [  1.38382 ],
       [ -0.591574]])

In [105]:
y_train[0]

19

In [106]:
# Transformer Encoder function with dropout and L2 regularization
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed-forward network with dropout and L2 regularization
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(res)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [107]:
# Transformer Decoder function
def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(res, enc_outputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res

    # Feed-forward network
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res


In [108]:
# Build the model
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0, l2_reg=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout, l2_reg)


    # Decoder
    decoder_input = keras.Input(shape=input_shape)
    dec = decoder_input
    for _ in range(num_transformer_blocks):
        dec = transformer_decoder(dec, x, head_size, num_heads, ff_dim, dropout, l2_reg)

    x = layers.Flatten()(x)
    dec = layers.Flatten()(dec)
    x = layers.Concatenate()([x, dec])

    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model([inputs, decoder_input], outputs)
    return model

In [109]:
# Define the hyperparameters
"""
# THIS IS THE PART YOU SHOULD PLAY AROUND WITH
# YOU MAY REDUCE THESE NUMBERS IF YOUR MACHINE IS OTHERWISE INCAPABLE OF RUNNING THE CODE"""

head_size = 4
num_heads = 4
ff_dim = 256
num_transformer_blocks = 4
mlp_units = [256]
dropout = 0.1
mlp_dropout = 0.1
initial_lr = 0.001
l2_reg=0.01

In [110]:
# Build the model
model = build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout, mlp_dropout)

In [111]:
# Create the optimizer with the initial learning rate
optimizer = Adam(learning_rate=initial_lr)

In [112]:
# Compile the model
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [113]:
# Create the ReduceLROnPlateau callback
lr_callback = ReduceLROnPlateau(factor=0.1, patience=50)

In [114]:
# Record the start time
start_time = time.time()

In [115]:
model.fit([x_train, x_train], y_train,
          batch_size=64,
          epochs=100,
          validation_data=([x_val, x_val], y_val),
          callbacks=[lr_callback])

Epoch 1/100
914/914 [==============================] - 97s 70ms/step - loss: 1.9367 - accuracy: 0.5469 - val_loss: 0.7606 - val_accuracy: 0.7463 - lr: 0.0010
Epoch 2/100
914/914 [==============================] - 62s 68ms/step - loss: 0.7403 - accuracy: 0.7307 - val_loss: 0.5582 - val_accuracy: 0.8147 - lr: 0.0010
Epoch 3/100
914/914 [==============================] - 61s 67ms/step - loss: 0.5214 - accuracy: 0.8167 - val_loss: 0.4218 - val_accuracy: 0.8619 - lr: 0.0010
Epoch 4/100
914/914 [==============================] - 58s 63ms/step - loss: 0.3820 - accuracy: 0.8683 - val_loss: 0.3056 - val_accuracy: 0.9065 - lr: 0.0010
Epoch 5/100
914/914 [==============================] - 58s 64ms/step - loss: 0.3086 - accuracy: 0.8927 - val_loss: 0.2616 - val_accuracy: 0.9128 - lr: 0.0010
Epoch 6/100
914/914 [==============================] - 61s 67ms/step - loss: 0.2599 - accuracy: 0.9094 - val_loss: 0.2225 - val_accuracy: 0.9262 - lr: 0.0010
Epoch 7/100
914/914 [==============================]

In [116]:
# Record the end time
end_time = time.time()

In [117]:
# Calculate the total running time
running_time = end_time - start_time
print("Total Running Time: {:.2f} seconds".format(running_time))

Total Running Time: 6282.37 seconds


In [118]:
# Evaluate the model
loss, accuracy = model.evaluate([x_test, x_test], y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

254/254 [==============================] - 5s 21ms/step - loss: 0.1232 - accuracy: 0.9512
Test loss: 0.12318335473537445, Test accuracy: 0.9512315392494202


In [119]:
# Save the trained model
model.save("my_model_miguel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
